In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day29-sklearn-pipelines/train.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
264,265,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.7500,NaN,Q
853,854,1,1,"Lines, Miss. Mary Conover",female,16.0,0,1,PC 17592,39.4000,D28,S
139,140,0,1,"Giglio, Mr. Victor",male,24.0,0,0,PC 17593,79.2000,B86,C
464,465,0,3,"Maisner, Mr. Simon",male,NaN,0,0,A/S 2816,8.0500,NaN,S
653,654,1,3,"O'Leary, Miss. Hanora ""Norah""",female,NaN,0,0,330919,7.8292,NaN,Q


In [3]:
df.shape

(891, 12)

In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                        df['Survived'],
                                                        test_size=0.2,
                                                        random_state=42)

In [7]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [9]:
#Imputation Transformer

trf1 = ColumnTransformer([
    ('imp_age',SimpleImputer(),[2]),
    ('imp_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

Note:
    While Using pipelines and transformers,
    use columns Indices instead of column names 
    Because it stores as a numpy array and numpy array doesn't contain column names

In [10]:
#one hot encoding

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [11]:
#scaling

trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

while using Feature Selection ,
we should use MinMaxScaler instead of StandardScaler

After onehotencoding the number of columns become 10 columns

In [12]:
#Feature Selection
from sklearn.feature_selection import SelectKBest,chi2

trf4 = SelectKBest(score_func=chi2,k=8)

In [13]:
#train the model
from sklearn.tree import DecisionTreeClassifier

trf5 = DecisionTreeClassifier()

### Create Pipeline

In [14]:
from sklearn.pipeline import Pipeline,make_pipeline

pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

### Pipeline vs make_pipeline

Pipeline requires naming of steps,make_pipeline does not.

(Same applicable to ColumnTransformer vs make_column_transformer)

In [15]:
#alternate syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

If there is a model present in the pipeline, Use fit

else there is no model training, only the preprocessing then use fit_transform

In [16]:
#train
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imp_age', SimpleImputer(),
                                                  [2]),
                                                 ('imp_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
      

In [20]:
# #Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [21]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imp_age', SimpleImputer(),
                                                  [2]),
                                                 ('imp_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000018B7FC42CA0>)),
                ('trf5', DecisionTreeClassifier())])

### Explore the Pipeline
In Exploration of the Pipelines,
we will learn visualizing the pipeline and post-mortem of pipeline

In [19]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('imp_age', SimpleImputer(), [2]),
                                 ('imp_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x0000018B7FC42CA0>),
 'trf5': DecisionTreeClassifier()}

In [22]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('imp_age', SimpleImputer(), [2]),
                                ('imp_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [24]:
pipe.named_steps['trf1'].transformers_

[('imp_age', SimpleImputer(), [2]),
 ('imp_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [28]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [29]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

### Predict

In [38]:
y_pred = pipe.predict(x_test)
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

### Cross Validation using Pipeline

In [40]:
#cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean()

0.6391214419383433

### GridSearch using Pipeline

In [45]:
#gridsearchcv
params = {
    'trf5__max_depth' : [1,2,3,4,5,None]
}

In the above code trf5__ double underscore is required to specify the parameters

In [46]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('imp_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imp_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x0000018B7FC42CA0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [47]:
grid.best_score_

0.6391214419383433

In [48]:
grid.best_params_

{'trf5__max_depth': 2}

### Exporting the Pipeline

In [49]:
#export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))